In [59]:
import pandas as pd
import numpy as np
import random

In [60]:
df = pd.read_csv (r'C:\Notebook scripts\Features_Variant_1.csv')

data_train=np.asarray(df.iloc[:,:])
#print(np.shape(data_train))

#shuffle before folding
np.random.shuffle(data_train)

In [61]:
#fold_1, fold_2, fold_3, fold_4, fold_5= np.array_split(data_train,5)
data_train_arr=np.array_split(data_train,5) #{[],[],[],[],[]}

y_train_arr=[]
x_train_arr=[]
for i in range(len(data_train_arr)):
    y_train_arr.append(data_train_arr[i][:,-1].reshape((-1, 1)) )
    x_train_arr.append(data_train_arr[i][:,:-1])


In [62]:
#потом нормализовать по ф-ле
mean_item=[]
std_item=[]
#print(x_train_arr[0][33,5])
for item in range(len(x_train_arr)):
    
    mean_item=np.mean(x_train_arr[item],axis=0)#(1,53)
    std_item=np.std(x_train_arr[item],axis=0)
    np.place(std_item, std_item==0, 1)#if std==0 then change it to 1 for division #std_item[std_item==0]=1 
    x_train_arr[item]=(x_train_arr[item]-mean_item)/std_item

In [63]:
# function to create a list containing mini-batches 
def create_mini_batches(X, y, batch_size): 
    mini_batches = [] 
    data = np.hstack((X, y)) 
    np.random.shuffle(data) 
    n_minibatches = data.shape[0] // batch_size 
    i = 0
  
    for i in range(n_minibatches + 1): 
        mini_batch = data[i * batch_size:(i + 1)*batch_size, :] 
        X_mini = mini_batch[:, :-1] 
        Y_mini = mini_batch[:, -1].reshape((-1, 1)) 
        mini_batches.append((X_mini, Y_mini)) 
    if data.shape[0] % batch_size != 0: 
        mini_batch = data[i * batch_size:data.shape[0]] 
        X_mini = mini_batch[:, :-1] 
        Y_mini = mini_batch[:, -1].reshape((-1, 1)) 
        mini_batches.append((X_mini, Y_mini)) 
    return mini_batches 

#что делать с омега0?-просто число но на самом деле вектор одинаковых чисел (8189,1) как и у размерность совпадает
def linear_regression_predict(X,omega,omega0):
    prod=X.dot(omega)#(8190,1)
    #print(np.shape(y+prod))
    return prod+omega0

In [64]:
def root_mean_square_error(X,omega,omega0,y_actual):
    y_predict=linear_regression_predict(X,omega,omega0)
    summands=np.power(y_actual-y_predict,2)
    #print(np.shape(summands))
    mse=np.sum(summands)/len(y_predict)
    return np.sqrt(mse)
   
def r_squared(X,omega,omega0,y_actual):
    y_predict=linear_regression_predict(X,omega,omega0)
    nom=np.sum(np.power(y_actual-y_predict,2))
    denom=np.sum(np.power(y_actual-np.mean(y_actual,axis=0),2))
    return 1-nom/denom

In [65]:
def batch_gradient_descent(X,y_train,learning_rate,num_iterations):
                                   # dim omega
    #omega = np.matrix([0.0 for i in range(num_parameters)])
    omega=np.random.sample((X.shape[1],1)) #np.zeros((X.shape[1],1))
    omega0=0
    cost = [] 
    # init omega
    #cost = [0.0 for i in range(num_iterations)]
    it=0
    for it in range(num_iterations): 
        N=len(X)
        error=y_train-linear_regression_predict(X,omega,omega0)
        omega=omega+learning_rate*2*((X.T).dot(error))/N
        omega0=omega0+learning_rate*2*np.sum(error)/N
        cost[it] = r_squared(X,omega,omega0,y_train)
    return omega,omega0,cost

def mini_batch_gradient_descent(X,y_train,learning_rate,num_epoches,batch_size):
                                   # dim omega
    #omega = np.matrix([0.0 for i in range(num_parameters)])
    omega=np.random.sample((X.shape[1],1)) #np.zeros((X.shape[1],1))
    omega0=0
    cost = [] 
    # init omega
    #cost = [0.0 for i in range(num_iterations)]
    it=0
    for it in range(num_epoches): 
        mini_batches = create_mini_batches(X, y_train, batch_size) 
        for mini_batch in mini_batches: 
            X_mini, y_mini = mini_batch 
            N=len(X_mini)
            error=y_mini-linear_regression_predict(X_mini,omega,omega0)
            omega=omega+learning_rate*2*((X_mini.T).dot(error))/N
            omega0=omega0+learning_rate*2*np.sum(error)/N
            cost.append(r_squared(X_mini,omega,omega0, y_mini)) 
    return omega,omega0,cost

In [66]:
x_train_1=(np.concatenate((x_train_arr[0],x_train_arr[2],x_train_arr[3],x_train_arr[4])))
y_train_1=(np.concatenate((y_train_arr[0],y_train_arr[2],y_train_arr[3],y_train_arr[4])))

omega_1,omega0_1,cost=mini_batch_gradient_descent(x_train_1,y_train_1,learning_rate = 0.001,num_epoches=10,batch_size=32)

r2_1=r_squared(x_train_arr[1],omega_1,omega0_1,y_train_arr[1])
rmse_1=root_mean_square_error(x_train_arr[1],omega_1,omega0_1,y_train_arr[1])

r2=[]
rmse=[]
omega=[]
omega0=[]
r2.insert(1,r2_1)
rmse.insert(1,rmse_1)
omega.insert(1,omega_1)
omega0.insert(1,omega0_1)
r2_1,rmse_1


(0.3330320742042894, 24.10272015144553)

In [67]:
x_train_2=(np.concatenate((x_train_arr[1],x_train_arr[0],x_train_arr[3],x_train_arr[4])))
y_train_2=(np.concatenate((y_train_arr[1],y_train_arr[0],y_train_arr[3],y_train_arr[4])))

omega_2,omega0_2,cost=mini_batch_gradient_descent(x_train_2,y_train_2,learning_rate = 0.001,num_epoches=10,batch_size=32)

r2_2=r_squared(x_train_arr[2],omega_2,omega0_2,y_train_arr[2])
rmse_2=root_mean_square_error(x_train_arr[2],omega_2,omega0_2,y_train_arr[2])
r2.insert(2,r2_2)
rmse.insert(2,rmse_2)
omega.insert(2,omega_2)
omega0.insert(2,omega0_2)
r2_2,rmse_2

(0.3600822344832417, 27.921513308173285)

In [68]:
x_train_0=(np.concatenate((x_train_arr[1],x_train_arr[2],x_train_arr[3],x_train_arr[4])))
y_train_0=(np.concatenate((y_train_arr[1],y_train_arr[2],y_train_arr[3],y_train_arr[4])))

omega_0,omega0_0,cost=mini_batch_gradient_descent(x_train_0,y_train_0,learning_rate = 0.001,num_epoches=10,batch_size=32)

r2_0=r_squared(x_train_arr[0],omega_0,omega0_0,y_train_arr[0])
rmse_0=root_mean_square_error(x_train_arr[0],omega_0,omega0_0,y_train_arr[0])
r2.insert(0,r2_0)
rmse.insert(0,rmse_0)
omega.insert(0,omega_0)
omega0.insert(0,omega0_0)
r2_0,rmse_0



(0.3305421136640012, 32.66239816128469)

In [69]:
x_train_3=(np.concatenate((x_train_arr[1],x_train_arr[2],x_train_arr[0],x_train_arr[4])))
y_train_3=(np.concatenate((y_train_arr[1],y_train_arr[2],y_train_arr[0],y_train_arr[4])))

omega_3,omega0_3,cost=mini_batch_gradient_descent(x_train_3,y_train_3,learning_rate = 0.001,num_epoches=10,batch_size=32)

r2_3=r_squared(x_train_arr[3],omega_3,omega0_3,y_train_arr[3])
rmse_3=root_mean_square_error(x_train_arr[3],omega_3,omega0_3,y_train_arr[3])
r2.insert(3,r2_3)
rmse.insert(3,rmse_3)
omega.insert(3,omega_3)
omega0.insert(3,omega0_3)
r2_3,rmse_3

(0.3004581714627371, 26.923860625799758)

In [70]:
x_train_4=(np.concatenate((x_train_arr[1],x_train_arr[2],x_train_arr[0],x_train_arr[3])))
y_train_4=(np.concatenate((y_train_arr[1],y_train_arr[2],y_train_arr[0],y_train_arr[3])))

omega_4,omega0_4,cost=mini_batch_gradient_descent(x_train_4,y_train_4,learning_rate = 0.001,num_epoches=25,batch_size=32)

r2_4=r_squared(x_train_arr[4],omega_4,omega0_4,y_train_arr[4])
rmse_4=root_mean_square_error(x_train_arr[4],omega_4,omega0_4,y_train_arr[4])
r2.insert(4,r2_4)
rmse.insert(4,rmse_4)
omega.insert(4,omega_4)
omega0.insert(4,omega0_4)
r2_4,rmse_4

(0.23388135757718598, 34.77459078377353)

In [71]:
omega,omega0
rmse,r2

([32.66239816128469,
  24.10272015144553,
  27.921513308173285,
  26.923860625799758,
  34.77459078377353],
 [0.3305421136640012,
  0.3330320742042894,
  0.3600822344832417,
  0.3004581714627371,
  0.23388135757718598])

In [72]:
np.mean(rmse)

29.277016606095362

In [73]:
np.mean(r2)

0.3115991902782911

In [78]:
np.mean(omega,axis=0),np.mean(omega0)

(array([[-3.16973910e-01],
        [-6.47711504e-01],
        [-1.18014532e+00],
        [-1.58015803e-01],
        [-6.82192440e-01],
        [ 1.09391026e+00],
        [ 9.24217916e-01],
        [ 1.95303076e-01],
        [ 2.33024236e+00],
        [ 1.21355668e-01],
        [-4.32635973e-01],
        [ 3.51682141e-01],
        [ 2.60876355e+00],
        [ 1.05257576e+00],
        [ 1.17298408e-01],
        [ 9.69712472e-01],
        [ 7.76183073e-01],
        [ 2.82646940e+00],
        [-4.01039607e-03],
        [-5.67092041e-01],
        [-1.53036040e+00],
        [-5.38651922e-01],
        [-1.13085901e+00],
        [-9.19226993e-02],
        [ 6.71979677e-01],
        [-4.36604854e-01],
        [ 2.89710585e-02],
        [-2.36920799e-01],
        [-2.45174142e-01],
        [-4.55100105e-01],
        [ 1.06076625e+01],
        [ 2.97070825e+00],
        [-1.86226241e+00],
        [ 6.41670696e+00],
        [-4.09434254e+00],
        [ 1.04392575e-01],
        [ 3.01785867e+00],
 

In [79]:
np.std(omega,axis=0),np.std(omega0)

(array([[0.71679107],
        [0.17066199],
        [0.30272207],
        [0.07987249],
        [0.61835669],
        [0.23830868],
        [0.54658257],
        [0.58381919],
        [0.23141935],
        [0.62034804],
        [0.3678269 ],
        [0.80565105],
        [0.35514059],
        [1.07140354],
        [0.30210496],
        [0.56641299],
        [0.37517487],
        [0.4462755 ],
        [0.43829577],
        [0.29843375],
        [0.41364893],
        [0.40422965],
        [0.46367623],
        [0.38135131],
        [0.27878848],
        [0.13138862],
        [0.65690186],
        [0.8882246 ],
        [0.84810506],
        [0.27723678],
        [0.33284244],
        [0.27376196],
        [0.59918041],
        [0.17645433],
        [0.17296804],
        [0.14806789],
        [0.56536354],
        [0.31652653],
        [0.0463535 ],
        [0.22235049],
        [0.1093785 ],
        [0.09994433],
        [0.18636447],
        [0.16768732],
        [0.12283306],
        [0

In [86]:
# initialise data of lists. 
data = {'1':[rmse[0],r2[0],omega0[0], omega[0]], 
        '2':[rmse[1],r2[1],omega0[1], omega[1]],
        '3':[rmse[2],r2[2],omega0[2], omega[2]],
        '4':[rmse[3],r2[3],omega0[3], omega[3]],
        '5':[rmse[4],r2[4],omega0[4], omega[4]],
        'E':[np.mean(rmse),np.mean(r2), np.mean(omega0),np.mean(omega,axis=0)],
        'SD':[np.std(rmse),np.std(r2), np.std(omega0),np.std(omega,axis=0)]} 
 
# Creates pandas DataFrame. 
df = pd.DataFrame(data, index =['RMSE', 'R^2', 'omega0', 'weights']) 
df

,1,2,3,4,5,E,SD
RMSE,32.6624,24.1027,27.9215,26.9239,34.7746,29.277,3.89441
R^2,0.330542,0.333032,0.360082,0.300458,0.233881,0.311599,0.0432041
omega0,7.1518,7.4492,7.26895,7.57462,7.2679,7.34249,0.150072
weights,"[[-1.5922402985042114], [-0.7891826195734165],...","[[-0.2996122782874196], [-0.5545957366298386],...","[[-0.13967213709313253], [-0.5314484880875563]...","[[-0.18105718045268585], [-0.45680968289535084...","[[0.6277123460463381], [-0.9065209921186184], ...","[[-0.3169739096582222], [-0.6477115038609561],...","[[0.7167910694636062], [0.1706619863852937], [..."


In [88]:
np.mean(linear_regression_predict(x_train_4,omega[4],omega0[4]))

7.26790018537433